In [1]:
import pandas as pd
import numpy as np
import json
import glob
import os

In [2]:
import glob
import os

import pandas as pd
import tiktoken

ENCODING = 'p50k_base'
enc = tiktoken.get_encoding(ENCODING)
assert enc.decode(enc.encode("hello world")) == "hello world"

def tokenize(text):
    return ' '.join(map(str, enc.encode(text, disallowed_special=())))

In [3]:
def get_combined_df(repo_dir):
    all_files = glob.glob(os.path.join(repo_dir, '*.parquet'))
    all_files.sort()
    all_dataframes = [pd.read_parquet(file) for file in all_files]
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    combined_df['commit_date'] = (combined_df['commit_date'].astype('int64') / 1e9).astype('int64')
    # replace NaN with empty string
    # combined_df.fillna('', inplace=True)
    return combined_df

In [4]:
# repo_dir = '../data/pytorch_pytorch/'
repo_dir = '../smalldata/fbr/'
fbdf = get_combined_df(repo_dir)

In [5]:
fbdf.shape

(69835, 14)

In [7]:
# get all rows with is_merge_request == True
fbdf[fbdf['is_merge_request'] == True].head()

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
15553,facebook,react,1451377284,51295bf7d014c3225f14e11c408da438a7f84f5f,Merge pull request #5741 from mziwisky/patch-1...,src/addons/transitions/ReactCSSTransitionGroup...,"/** * Copyright 2013-2015, Facebook, Inc. * ...",713401f2eb4b892f872ec61b15fdb597c4eeaae6,<NA>,"/** * Copyright 2013-2015, Facebook, Inc. * ...","@@ -19,10 +19,6 @@ var ReactTransitionEvents =...",modified,True,js
15555,facebook,react,1451051613,53dabe748c6817cbb7b43f975e24b61331011077,Merge pull request #5714 from gaearon/passthro...,src/renderers/dom/client/__tests__/ReactDOMSVG...,"/** * Copyright 2013-2015, Facebook, Inc. * ...",82fe64a4564238f8923020f6f4ab4864d1459533,<NA>,"/** * Copyright 2013-2015, Facebook, Inc. * ...","@@ -21,6 +21,45 @@ describe('ReactDOMSVG', fun...",modified,True,js
15556,facebook,react,1451051613,53dabe748c6817cbb7b43f975e24b61331011077,Merge pull request #5714 from gaearon/passthro...,src/renderers/dom/shared/DOMPropertyOperations.js,"/** * Copyright 2013-2015, Facebook, Inc. * ...",82fe64a4564238f8923020f6f4ab4864d1459533,<NA>,"/** * Copyright 2013-2015, Facebook, Inc. * ...","@@ -124,6 +124,31 @@ var DOMPropertyOperations...",modified,True,js
15557,facebook,react,1451051613,53dabe748c6817cbb7b43f975e24b61331011077,Merge pull request #5714 from gaearon/passthro...,src/renderers/dom/shared/ReactDOMComponent.js,"/** * Copyright 2013-2015, Facebook, Inc. * ...",82fe64a4564238f8923020f6f4ab4864d1459533,<NA>,"/** * Copyright 2013-2015, Facebook, Inc. * ...","@@ -647,6 +647,8 @@ ReactDOMComponent.Mixin = ...",modified,True,js
15558,facebook,react,1451051613,53dabe748c6817cbb7b43f975e24b61331011077,Merge pull request #5714 from gaearon/passthro...,src/renderers/dom/shared/ReactDOMDebugTool.js,"/** * Copyright 2013-2015, Facebook, Inc. * ...",82fe64a4564238f8923020f6f4ab4864d1459533,<NA>,"/** * Copyright 2013-2015, Facebook, Inc. * ...","@@ -12,6 +12,8 @@ 'use strict'; var ReactD...",modified,True,js


In [8]:
# get number of unique merge requests (by seeing unique commit_id)
fbdf[fbdf['is_merge_request'] == True]['commit_id'].nunique()

1666

In [6]:
fbdf.head()

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
0,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/run_test.py,#!/usr/bin/env python3 import argparse import...,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,#!/usr/bin/env python3 import argparse import...,"@@ -259,8 +259,6 @@ CORE_TEST_LIST = [ ""t...",modified,False,py
1,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/test_ops.py,"# Owner(s): [""high priority""] from collection...",7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,"# Owner(s): [""high priority""] from collection...","@@ -1,25 +1,28 @@ # Owner(s): [""high priority...",modified,False,py
2,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/test_ops_jit.py,<NA>,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,"# Owner(s): [""high priority""] from functools ...",<NA>,deleted,False,py
3,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",torch/autograd/gradcheck.py,import torch from torch.types import _TensorOr...,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,import torch from torch.types import _TensorOr...,"@@ -504,7 +504,7 @@ workarounds. The workaroun...",modified,False,py
4,pytorch,pytorch,1647459022,7cf9b942daa824267ec3902f1026b3e6353561ff,Move test ops gradients and test ops jit to se...,test/run_test.py,#!/usr/bin/env python3 import argparse import...,e084e266da261fb25742cb4caa60ce9e79181e3a,<NA>,#!/usr/bin/env python3 import argparse import...,"@@ -259,6 +259,8 @@ CORE_TEST_LIST = [ ""t...",modified,False,py


In [7]:
# show counts by status
fbdf.groupby('status').count()

/var/folders/00/mwsht4n55dl_7kj9p52w9j940000gn/T/ipykernel_33757/1818841972.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fbdf.groupby('status').count()


,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,is_merge_request,file_extension
status,,,,,,,,,,,,,
added,34,34,34,34,34,34,34,0,0,0,0,34,34
deleted,14,14,14,14,14,14,0,14,0,14,0,14,14
modified,560,560,560,560,560,560,560,560,0,560,560,560,560
renamed,9,9,9,9,9,9,9,9,9,9,9,9,9


In [99]:
# show counts by status
fbdf.groupby('status').count()

/var/folders/00/mwsht4n55dl_7kj9p52w9j940000gn/T/ipykernel_38974/1818841972.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fbdf.groupby('status').count()


,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,is_merge_request,file_extension
status,,,,,,,,,,,,,
added,5588,5588,5588,5588,5588,5588,5588,0,0,0,0,5588,5588
deleted,2656,2656,2656,2656,2656,2656,0,2656,0,2656,0,2656,2656
modified,57451,57451,57451,57451,57451,57451,57451,57451,0,57451,57450,57451,57451
renamed,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140


In [63]:
# see sample of status == 'renamed'
fbdf[fbdf['status'] == 'renamed'].head()

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
21,facebook,react,1696026245,c7ba8c098889b6dc47fa9c807bbba3975a658584,"Enforce that the ""react-server"" build of ""reac...",packages/react/src/ReactSharedInternalsClient.js,"/** * Copyright (c) Meta Platforms, Inc. and ...",d900fadbf9017063fecb2641b7e99303b82a6f17,packages/react/src/ReactSharedInternals.js,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,33 @@ +/** + * Copyright (c) Meta P...",renamed,False,js
126,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-esm/src/ReactFlightC...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-esm/src/ReactFlightC...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,120 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
131,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,162 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
132,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,234 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
462,facebook,react,1693307366,8fbd3079425eaff85ec1b52b0eefecdd44ba7289,refactor: refactored devtools browser extensio...,packages/react-devtools-extensions/src/content...,"/** * Copyright (c) Meta Platforms, Inc. and ...",f47956239f6892a1e805af1fc3f1b2fb0f66beca,packages/react-devtools-extensions/src/backend...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,179 @@ +/** + * Copyright (c) Meta ...",renamed,False,js


In [8]:
print(fbdf.query("commit_id == 'c7ba8c098889b6dc47fa9c807bbba3975a658584' & file_path == 'packages/react-server-dom-turbopack/src/__tests__/ReactFlightDOMNode-test.js'")['cur_file_content'].values[0])

/**
 * Copyright (c) Meta Platforms, Inc. and affiliates.
 *
 * This source code is licensed under the MIT license found in the
 * LICENSE file in the root directory of this source tree.
 *
 * @emails react-core
 */

'use strict';

// Don't wait before processing work on the server.
// TODO: we can replace this with FlightServer.act().
global.setImmediate = cb => cb();

let clientExports;
let turbopackMap;
let turbopackModules;
let turbopackModuleLoading;
let React;
let ReactDOMServer;
let ReactServerDOMServer;
let ReactServerDOMClient;
let Stream;
let use;

describe('ReactFlightDOMNode', () => {
  beforeEach(() => {
    jest.resetModules();

    // Simulate the condition resolution
    jest.mock('react', () => require('react/react.shared-subset'));
    jest.mock('react-server-dom-turbopack/server', () =>
      require('react-server-dom-turbopack/server.node'),
    );
    ReactServerDOMServer = require('react-server-dom-turbopack/server');

    const TurbopackMock = require('./utils/Tu

In [26]:
fbdf.query("commit_id == '7e8e320b34b02e7d89d88b3a6c3059d5d23a2aa7'")

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension


In [15]:
fbdf.query("commit_id == '232faeacf8ada21d4dbbcb9cd296a67a99dccf9e'")

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
2871,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/run_test.py,#!/usr/bin/env python3 import argparse import...,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,#!/usr/bin/env python3 import argparse import...,"@@ -259,8 +259,6 @@ CORE_TEST_LIST = [ ""t...",modified,False,py
2872,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/test_ops.py,"# Owner(s): [""high priority""] from collection...",7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,"# Owner(s): [""high priority""] from collection...","@@ -1,25 +1,28 @@ # Owner(s): [""high priority...",modified,False,py
2873,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",test/test_ops_jit.py,<NA>,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,"# Owner(s): [""high priority""] from functools ...",<NA>,deleted,False,py
2874,pytorch,pytorch,1647461303,232faeacf8ada21d4dbbcb9cd296a67a99dccf9e,"Revert ""Move test ops gradients and test ops j...",torch/autograd/gradcheck.py,import torch from torch.types import _TensorOr...,7906ac40dd1bbe3200fda334d645970fb5f58c61,<NA>,import torch from torch.types import _TensorOr...,"@@ -504,7 +504,7 @@ workarounds. The workaroun...",modified,False,py


In [16]:
print(fbdf.query("commit_id == '232faeacf8ada21d4dbbcb9cd296a67a99dccf9e' & file_path=='test/run_test.py'")['diff'].values[0])

@@ -259,8 +259,6 @@ CORE_TEST_LIST = [
     "test_modules",
     "test_nn",
     "test_ops",
-    "test_ops_gradients",
-    "test_ops_jit",
     "test_torch"
 ]
 


In [38]:
from "../src/bm25" import search

SyntaxError: invalid syntax (2229958662.py, line 1)

In [35]:
fbdf.query("commit_id == '84cacaf5b6e5f6c5c840bb9444f88a299c138301'")['commit_message'].values[0]

'Merge pull request #717 from syranide/testkeytransfer\n\nAdd unit test for transferPropsTo, "key" should never transfer'

In [19]:
print(fbdf.query("commit_id == '7cf9b942daa824267ec3902f1026b3e6353561ff' & file_path=='test/run_test.py'")['diff'].values[0])

@@ -259,6 +259,8 @@ CORE_TEST_LIST = [
     "test_modules",
     "test_nn",
     "test_ops",
+    "test_ops_gradients",
+    "test_ops_jit",
     "test_torch"
 ]
 


In [88]:
fbdf.query("commit_id == '069d7d7e72e73c9b8c0d6b22e77073e227c72a60'")

,owner,repo_name,commit_date,commit_id,commit_message,file_path,previous_commit_id,previous_file_content,cur_file_content,diff,status,is_merge_request,file_extension


In [27]:
import git

In [28]:
fbRepo = git.Repo('../repos/facebook_react/')

In [42]:
# get commit 069d7d7e72e73c9b8c0d6b22e77073e227c72a60

commit = fbRepo.commit('84cacaf5b6e5f6c5c840bb9444f88a299c138301')

In [44]:
fbdf.query("commit_id == '84cacaf5b6e5f6c5c840bb9444f88a299c138301'")

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
27820,facebook,react,1388199296,84cacaf5b6e5f6c5c840bb9444f88a299c138301,Merge pull request #717 from syranide/testkeyt...,src/core/__tests__/ReactPropTransferer-test.js,"/** * Copyright 2013 Facebook, Inc. * * Lic...",72fd24662e999986ddd3b27793e02f5c1ba56792,<NA>,"/** * Copyright 2013 Facebook, Inc. * * Lic...","@@ -110,25 +110,26 @@ describe('ReactPropTrans...",modified,True,js


In [41]:
from pyserini.index.lucene import IndexReader
from pyserini.search.lucene import LuceneSearcher

/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
def search(searcher, query, query_date, k=1000):
    hits = searcher.search(tokenize(query), k)
    unix_date = query_date
    filtered_hits = [hit for hit in hits if int(json.loads(hit.raw)["commit_date"]) < unix_date]
    return filtered_hits

In [45]:
searcher = LuceneSearcher('../smalldata/fbr/index_commit_tokenized/')

In [60]:
query_msg = fbdf.query("commit_id == '84cacaf5b6e5f6c5c840bb9444f88a299c138301'")['commit_message'].values[0]
query_date = fbdf.query("commit_id == '84cacaf5b6e5f6c5c840bb9444f88a299c138301'")['commit_date'].values[0]
query_msg, query_date

('Merge pull request #717 from syranide/testkeytransfer\n\nAdd unit test for transferPropsTo, "key" should never transfer',
 1388199296)

In [65]:
obj

{'id': '02e47ebd008806008f29cf1f058614365c8035f9',
 'contents': '13102 469 2834 2581 1303 50055 422 827 2596 485 14 77 2088 88 39437 198 198 39437 2964 862 2514 815 1239 4351 262 366 2539 1 3119',
 'repo_name': 'react',
 'file_path': 'src/core/ReactPropTransferer.js',
 'commit_date': 1388033190}

In [75]:
def reverse_tokenize(text):
    # text = json.loads(text)
    # print(list(text['contents'].split(' ')))
    return enc.decode([int(i) for i in text.split(' ')])
    # return string
    # return json.dumps(text, indent=2)

In [76]:
reverse_tokenize(obj['contents'])

'Merge pull request #713 from syranide/nokeytransfer\n\ntransferPropsTo should never transfer the "key" property'

In [78]:
query_date

1388199296

In [77]:
max_date = 0
test_hits = search(searcher, query_msg, query_date)
for i in range(len(test_hits)):
    # print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    # print with repo name and file name
    obj = json.loads(test_hits[i].raw)
    # print(obj)
    commit_date = int(obj["commit_date"])
    commit_msg = reverse_tokenize(obj["contents"])
    max_date = max(max_date, commit_date)
    print(f'{i+1:2} {test_hits[i].docid:4} {test_hits[i].score:.5f} {obj["repo_name"]}/{obj["file_path"]} {commit_date} {commit_msg}')

 1 02e47ebd008806008f29cf1f058614365c8035f9 53.43890 react/src/core/ReactPropTransferer.js 1388033190 Merge pull request #713 from syranide/nokeytransfer

transferPropsTo should never transfer the "key" property
 2 f9947dec2aa5efaf4b3e74392415fff693e8f189 38.66000 react/src/core/__tests__/ReactPropTransferer-test.js 1388072559 Add unit test for transferPropsTo, "key" should never transfer

 3 9e0987cd9bcb12c6ab69fef8fa1f6136238fb869 28.79720 react/src/event/synthetic/SyntheticKeyboardEvent.js 1387825872 Merge pull request #607 from syranide/html5key

Polyfill and normalize HTML5 "key", deprecates which and keyCode
 4 f627bc52b0c823368df9ed662587de5c2e33a58e 25.71629 react/src/core/__tests__/ReactIdentity-test.js 1388193256 Merge pull request #714 from syranide/escapekey

Escape component keys used in reactid
 5 f627bc52b0c823368df9ed662587de5c2e33a58e 25.71629 react/src/utils/__tests__/ReactChildren-test.js 1388193256 Merge pull request #714 from syranide/escapekey

Escape component ke

In [53]:
max_date

1388033190

In [63]:
max_date <= query_date

True